In [11]:
from transformers import AutoModel, AutoTokenizer, BertModel, BertTokenizer

model = BertModel.from_pretrained("bert-base-cased")

In [12]:
model.embeddings

BertEmbeddings(
  (word_embeddings): Embedding(28996, 768, padding_idx=0)
  (position_embeddings): Embedding(512, 768)
  (token_type_embeddings): Embedding(2, 768)
  (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
  (dropout): Dropout(p=0.1, inplace=False)
)

In [13]:
import torch
from torch import nn, Tensor
import math


class PositionalEncoding(nn.Module):
    def __init__(self, embed_size: int, dropout: float = 0, max_len: int = 512):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, embed_size, 2) * (-math.log(10000.0) / embed_size))
        pe = torch.zeros(max_len, 1, embed_size)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer("pe", pe)

    def forward(self, x: Tensor) -> Tensor:
        # (B, seq_len, embed_size) -> (B, seq_len, embed_size)
        x = x + self.pe[: x.size(0)]
        return self.dropout(x)

In [14]:
tokenizer = BertTokenizer.from_pretrained("bert-base-cased")
embedding = nn.Embedding(len(tokenizer), 300)
pos_embedding = PositionalEncoding(300)
print(pos_embedding)

PositionalEncoding(
  (dropout): Dropout(p=0, inplace=False)
)


In [15]:
sentences = [
    "This is the first sentences",
    "I've been meaning to write for ages and finally today",
]

inputs = tokenizer(sentences, return_tensors="pt", padding=True)

In [16]:
embedded = embedding(inputs["input_ids"])
print(embedded.shape)
embedded_pos = pos_embedding(embedded)
print(embedded_pos.shape)

torch.Size([2, 14, 300])
torch.Size([2, 14, 300])


: 